#### ספריות


In [1]:
import os
import sys
import pandas as pd
import geopandas as gpd

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### העלאת משתנים להרצת הקוד


In [3]:
cwd = os.getcwd()

create_forecast_basic_future_folder_path = os.path.dirname(cwd)

create_forecast_basic_folder_path = os.path.dirname(create_forecast_basic_future_folder_path)

sys.path.append(create_forecast_basic_folder_path)

### פונקציות


### פונקציות גלובליות


In [4]:
from global_functions import get_newest_date_file, up_load_df, up_load_gdb, up_load_shp, drop_geo,change_cols_names, move_old_forcasts

In [5]:
forecast_version_folder_location=r'{}\outputs'.format(create_forecast_basic_folder_path)

forecast_version_base_year_folder_location=r'{}\BASE_YEAR'.format(forecast_version_folder_location)

In [6]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [7]:
TAZ_V4_date = get_newest_date_file(create_forecast_basic_folder_path, 'with_geo_info')

taz=up_load_shp(r'{}\background_files\TAZ_V4_{}_with_geo_info.shp'.format(create_forecast_basic_folder_path, TAZ_V4_date))

In [8]:
file_year = []
sen=['jtmt','iplan','bau']
sen_for_file_name=['jtmt','iplan_changed','bau_changed']
path=['JTMT\Intermediates','iplan\Intermediates\change_pop','bau\Intermediates\change_pop']

for sf, p in zip(sen_for_file_name, path):
    folder_path=r'{}\{}'.format(create_forecast_basic_future_folder_path, p)
    forecast_pop_date = get_newest_date_file(folder_path, 'pop_2025')
    file_year.append(forecast_pop_date)

In [9]:
sen_for_file_name=['jtmt','iplan_changed','bau_changed']
sen=['jtmt','iplan','bau']
path=['JTMT\Intermediates','iplan\Intermediates\change_pop','bau\Intermediates\change_pop']
year=['2025','2030','2035','2040','2045','2050']

for sf,s,p,fy in zip(sen_for_file_name,sen,path,file_year):
        for y in year:                        
                folder_path=r'{}\{}'.format(create_forecast_basic_future_folder_path, p)

                file_name='{}_pop_{}_{}'.format(fy,y,sf)

                df=up_load_df(folder_path,file_name)

                col_need=list(df)[1:20]
                
                locals()['forecast_pop_{}_{}'.format(y,s)]=df[col_need]

In [10]:
forecast_version_date = get_newest_date_file(forecast_version_base_year_folder_location, '2020_jtmt_forcast_full')

file_name = r'2020_jtmt_forcast_full_{}'.format(forecast_version_date)

forecast_2020=pd.read_excel(r'{}\{}.xlsx'.format(forecast_version_base_year_folder_location, file_name))

In [11]:
new_column_names = {'aprt_20': 'aprt','Yeshiva':'student_yeshiva'}
forecast_2020=forecast_2020.rename(columns=new_column_names)

In [12]:
col_constant=['Taz_num',
 'Taz_name',
 'jeru_metro',
 'in_jerusal',
 'SCHN_NAME',
 'ENG_NAME_n',
 'zonetype',
 'Sug_Muni',
 'CR_PNIM',
 'highBusine',
 'SCHOOLDIST',
 'Urban',
 'EIProp',
 'poly_puma',
 'F3',
 'F2',
 'F1',
 'ieold',
 'IEProp',
 'perScaled',
 'FreeBuffer',
 'PaidBuffer',
 'Rest_EmpBu',
 'searchtime',
 'walktime',
 'cost',
 'Agg_taz_nu',
 'area',
 'CITYCODE1',
 'CITYCODE2',
 'CITYCODE3',
 'CITYCODE4',
 'codeseq',
 'codeseqCon',
 'county',
 'majunivenr',
 'parktot',
 'superZone',
 'Taz1',
 'slope',
 'REGION',
 'yosh',
 'jerusalem_']

In [13]:
forecast_version_date = get_newest_date_file(forecast_version_base_year_folder_location, '2020_jtmt_forcast_full')

file_name = r'2020_jtmt_forcast_full_{}'.format(forecast_version_date)

taz_geo_info_constant=pd.read_excel(r'{}\{}.xlsx'.format(forecast_version_base_year_folder_location, file_name))[col_constant]

In [14]:
col_pre_uni=['Taz_num','sector_for_chinuc','pop_20_just_from_aprt','pop_25_just_from_aprt']
short_sector=['SE','AR','UO']
chinuch_sector=['Jewish','Arab','U_Orthodox']
for s,c in zip(short_sector,chinuch_sector):
    df=forecast_2020[col_pre_uni].query('sector_for_chinuc==@c')
    locals()['Univ_{}_pre_from_pop_20_30'.format(s)]= forecast_2020['Univ_{}'.format(s)].sum().item()/(df.sum()[['pop_20_just_from_aprt']].item()+df.sum()[['pop_25_just_from_aprt']].item())

In [15]:

df=forecast_2020.query('main_secto=="U_Orthodox"')
pop_15_20_just_from_aprt_ou=df.sum()[['pop_15_just_from_aprt']].item()+df.sum()[['pop_20_just_from_aprt']].item()

student_yeshiva=forecast_2020.student_yeshiva.sum().item()
student_seminar=forecast_2020.Seminar.sum().item()

student_yeshiva_pre_from_age_15_20=student_yeshiva/pop_15_20_just_from_aprt_ou
student_seminar_pre_from_age_15_20=student_seminar/pop_15_20_just_from_aprt_ou


In [16]:
EMP_kibolet=up_load_gdb(r'{}\background_files\EMP_kibolet.gdb'.format(create_forecast_basic_folder_path),'EMP_kibolet')

EMP_kibolet=EMP_kibolet.fillna(0)

EMP_kibolet['emp_kayim_no_palestinians']=EMP_kibolet['kayim_emp']-EMP_kibolet['Palestinians']

EMP_kibolet['emp_current']=EMP_kibolet['emp_kayim_no_palestinians']


col=[ 'F2025',
 'F2030',
 'F2035',
 'F2040',
 'F2045',
 'F2050']

EMP_kibolet[col]=EMP_kibolet[col].apply(lambda x: x * (EMP_kibolet['kibolt']-EMP_kibolet['kayim_emp']))

EMP_kibolet[col]=EMP_kibolet[col].cumsum(axis=1)

EMP_kibolet[col]=EMP_kibolet[col].apply(lambda x: x +EMP_kibolet['emp_kayim_no_palestinians'])

emp_not_okev_current=EMP_kibolet['emp_kayim_no_palestinians'].sum().item()

In [17]:
TAZ_V4_date = get_newest_date_file(create_forecast_basic_folder_path, 'with_geo_info')

taz=up_load_shp(r'{}\background_files\TAZ_V4_{}_with_geo_info.shp'.format(create_forecast_basic_folder_path, TAZ_V4_date))[['Taz_num','agri', 'Indus', 'Com_hotel', 'Business', 'Public','geometry']]

In [18]:


taz['taz_area']=taz.area

EMP_kibolet['emp_area']=EMP_kibolet.area

EMP_kibolet['ID']=EMP_kibolet.index


EMP_kibolet_by_taz=gpd.overlay(taz[['Taz_num','taz_area','geometry']],EMP_kibolet[['ID','emp_area','geometry']])

EMP_kibolet_by_taz['small_area']=EMP_kibolet_by_taz.area

EMP_kibolet_by_taz['pre_small_area_emp']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['emp_area']

EMP_kibolet_by_taz['pre_small_area_taz']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['taz_area']

EMP_kibolet_by_taz=EMP_kibolet_by_taz.query('(pre_small_area_taz >0.7) | (pre_small_area_emp >0.1)')[['ID','Taz_num','geometry']]

EMP_kibolet_by_taz['small_area']=EMP_kibolet_by_taz.area

EMP_kibolet_by_taz=EMP_kibolet_by_taz.set_index('ID')

EMP_kibolet_by_taz['id_area_for_pre']=drop_geo(EMP_kibolet_by_taz).groupby(by='ID').sum()['small_area']

EMP_kibolet_by_taz['prec_from_id']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['id_area_for_pre']


EMP_kibolet['geometry_buff']=EMP_kibolet.buffer(250)


C:\Users\dpere\AppData\Local\Temp\ipykernel_31032\1632213763.py:8: UserWarning: `keep_geom_type=True` in overlay resulted in 287 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  EMP_kibolet_by_taz=gpd.overlay(taz[['Taz_num','taz_area','geometry']],EMP_kibolet[['ID','emp_area','geometry']])


In [19]:
city_muni=['מודיעין עילית','בית שמש','ירושלים','מודיעין - מכבים - רעות']

In [20]:
#okev_factor=up_load_df(r'{}\current\Intermediates'.format(create_forecast_basic_folder_path),'okev_factor').set_index('sector')

df=up_load_df(r'{}\background_files'.format(create_forecast_basic_folder_path),'Labor_Force_Participation_Rate_Forecast_by_Sector_in_Jerusalem_until_2050')#.set_index('sector')
# Reshape the data using melt
work_factor = pd.melt(df, id_vars=['age', 'gender', 'sector'], var_name='year', value_name='value')

# Adjust the column order to match your target structure
work_factor = work_factor[['sector', 'value', 'age', 'gender', 'year']].set_index('sector')

In [21]:

col=['Taz_num','agri', 'Indus', 'Com_hotel', 'Business', 'Public']

emp_category_type=taz[col].set_index('Taz_num')


In [22]:
pre_woman=0.5

pre_man=1-pre_woman

work_age=[ 'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60']

under_work_age=[ 'pop_15', 'pop_20']

over_work_age=[ 'pop_65', 'pop_70', 'pop_75up']


In [23]:
folder_path=r'{}\background_files'.format(cwd)

taz_sector_change=up_load_df(folder_path,'230704_taz_num_convert_from_jewish_to_u_orthodox')
lst_change_sector=list(taz_sector_change.Taz_num)

In [24]:
emp_Education_per_uni_student=0.18
emp_education_per_student=7
unvi_in_dorms=1.5
pre_commuting_to_jtmt_area=1.06

sen=['jtmt','iplan','bau']

move_old_forcasts(sen,forecast_version_folder_location)

In [25]:
with open(r'{}\current\Intermediates\emp_in_jtmt_factor_to_emp_in_taz.txt'.format(create_forecast_basic_folder_path), 'r') as f:
    x=f.read()
    from_emp_to_place_of_work = float(x)

In [26]:
sen=['jtmt','iplan','bau']

for s in sen:

    #print('seniro:',s)

    if s!='jtmt':
        
        if s=='iplan':
            condtion=(forecast_2020['Taz_num'].isin(lst_change_sector))&(forecast_2020['zonetype']=='Judea and Samaria')
        elif s=='bau':
            condtion=(forecast_2020['Taz_num'].isin(lst_change_sector))
        forecast_2020.loc[condtion,['main_secto','chinuch_sector']]='U_Orthodox'

        
    col=['Taz_num',
        'Muni_Heb',
        'commuting',
        'main_secto','sector_for_chinuc','jew',
        'TOA1','TOA2','TOA3',
        'Univ_SE','Univ_AR','Univ_UO',
        'student_dorms','univ','emp_uni',
        'student_yeshiva','emp_from_Yeshiva_student',
        'yeshiva_dorms_pop_15',
        'yeshiva_dorms_pop_20',
        'yeshiva_dorms_pop_25',
        'yeshiva_dorms_pop_sum',
        'Seminar_dorms_pop_15',
        'Seminar_dorms_pop_20',
        'Seminar_dorms_pop_25',
        'Seminar','emp_from_Seminar_student']
    
    forecast_current=forecast_2020[col]

    years=['2025','2030','2035','2040','2045','2050']


    #print('univ before:',forecast_current['univ'].sum())

    



    for y in years:

        #print('year:',y)

        forecast=pd.merge(forecast_current,locals()['forecast_pop_{}_{}'.format(y,s)],on="Taz_num",how='right')

            
        forecast['student_demand_TOA1']=forecast['pop_5']/5*4+forecast['pop_10']/5*2
        forecast['student_demand_TOA2']=forecast['pop_10']/5*3
        forecast['student_demand_TOA3']=forecast['pop_15']/5*4


        demad_for_students_by_muni_and_sector=forecast.pivot_table(index=('Muni_Heb','sector_for_chinuc'),aggfunc='sum')[['student_demand_TOA1','student_demand_TOA2','student_demand_TOA3','TOA1','TOA2','TOA3']]
        cols_to_replace = ['TOA1', 'TOA2', 'TOA3']
        for col in cols_to_replace:
            diff_col_name = f'add_from_demad_{col}_muni_sector'
            demad_for_students_by_muni_and_sector[diff_col_name] = (demad_for_students_by_muni_and_sector[f'student_demand_{col}'] - demad_for_students_by_muni_and_sector[col]).clip(lower=0)


        # Reset index
        demad_for_students_by_muni_and_sector.reset_index(inplace=True)

        
        col=['Muni_Heb',
        'sector_for_chinuc',
        'add_from_demad_TOA1_muni_sector',
        'add_from_demad_TOA2_muni_sector',
        'add_from_demad_TOA3_muni_sector']
                           

        forecast=forecast.merge(demad_for_students_by_muni_and_sector[col],on=['Muni_Heb','sector_for_chinuc'])
        forecast=forecast.fillna(0)

        for i in range(1, 4):
            forecast[f'student_TOA{i}_to_fill_demand'] = (forecast[f'student_demand_TOA{i}'] - forecast[f'TOA{i}']).clip(lower=0)

    
        grouped_sum = forecast.groupby(['Muni_Heb', 'sector_for_chinuc']).sum().reset_index()

        for i in range(1, 4):  
            forecast = forecast.merge(grouped_sum[['Muni_Heb', 'sector_for_chinuc', f'student_TOA{i}_to_fill_demand']],
                                    on=['Muni_Heb', 'sector_for_chinuc'], suffixes=('', '_sum'))

        forecast=forecast.fillna(0)

        #להוסיף תלמידים בעקבות ביקוש
        for i in range(1, 4): 
            forecast[f'student_to_fill_TOA{i}_demand_pre'] = forecast[f'student_TOA{i}_to_fill_demand'] / forecast[f'student_TOA{i}_to_fill_demand_sum']
            forecast[f'add_to_TOA{i}_student_current'] = forecast[f'student_to_fill_TOA{i}_demand_pre'] * forecast[f'add_from_demad_TOA{i}_muni_sector']
            forecast[f'TOA{i}'] += forecast[f'add_to_TOA{i}_student_current']


        forecast['student']=forecast['TOA1']+forecast['TOA2']+forecast['TOA3']

        #המטרה לפצל את תלמידי שכבה לפי מגזר של האזור תנועה
        sector_for_loop=['SE','AR','UO']
        sector_for_chinuc_for_loop=[ 'Jewish','Arab', 'U_Orthodox']
        num_for_loop=['A1','A2','A3']

        for sc,sl in zip(sector_for_chinuc_for_loop,sector_for_loop):
            for n in num_for_loop:
                forecast.loc[forecast['sector_for_chinuc']==sc,'{}{}'.format(sl,n)]=forecast['TO{}'.format(n)]

        

        forecast['emp_from_student']=forecast['student']/emp_education_per_student


        #מה האחוז של כל מוסד מכלל הסטודנטים של המגזר
        univ_columns = ['Univ_SE', 'Univ_AR', 'Univ_UO']

        for col in univ_columns:
            forecast[f'uni_students_pre_{col}'] = forecast[col] / forecast[col].sum().item()




        #כמה סטודנטים מכל מגזר בחומש הזה

        sector_population = forecast.groupby('sector_for_chinuc')[['pop_20', 'pop_25']].sum()

        uni_SE_students = Univ_SE_pre_from_pop_20_30 * (sector_population.loc['Jewish', 'pop_20'] + sector_population.loc['Jewish', 'pop_25'])
        uni_AR_students = Univ_AR_pre_from_pop_20_30 * (sector_population.loc['Arab', 'pop_20'] + sector_population.loc['Arab', 'pop_25'])
        uni_UO_students = Univ_UO_pre_from_pop_20_30 * (sector_population.loc['U_Orthodox', 'pop_20'] + sector_population.loc['U_Orthodox', 'pop_25'])

        #print('uni_SE_students,uni_AR_students,uni_UO_students')

        #print(uni_SE_students,uni_AR_students,uni_UO_students)

        #print(uni_SE_students+uni_AR_students+uni_UO_students)




        #חלוקת הסטודנטים למוסדות
        sector_for_loop=['SE','AR','UO']

        for sl in sector_for_loop:

            forecast['add_Univ_{}_students'.format(sl)]=forecast['uni_students_pre_Univ_{}'.format(sl)]*locals()['uni_{}_students'.format(sl)]-forecast['Univ_{}'.format(sl)]

            forecast['Univ_{}'.format(sl)]=forecast['Univ_{}'.format(sl)]+forecast['add_Univ_{}_students'.format(sl)]


            

        forecast['current_univ']=forecast['univ']

        #print('current_univ:',forecast['current_univ'].sum())


        forecast['univ']=forecast['Univ_SE']+forecast['Univ_AR']+forecast['Univ_UO']

        #print('univ new:',forecast['univ'].sum())





        #חישוב תוספת סטודנטים בכל מוסד  בשביל תוספת התעסוקה במוסד
        forecast['add_uni_students']=forecast['univ']-forecast['current_univ']      
        forecast['emp_uni']=forecast['add_uni_students']*emp_Education_per_uni_student+forecast['emp_uni']

        #יצירת מקדם מעונות לסטודנטים

        total_dorm_students = forecast['student_dorms'].sum()
        total_current_univ_students = forecast['current_univ'].sum()

        dorms_vs_uni_students = total_dorm_students / total_current_univ_students
        growth_dorms = (forecast['univ'].sum() * dorms_vs_uni_students) / total_dorm_students

        forecast['student_dorms'] *= growth_dorms


        
        total_student_yeshiva=forecast.query('main_secto=="U_Orthodox"')[['pop_20','pop_15']].sum().sum()*student_yeshiva_pre_from_age_15_20
        total_student_seminar=forecast.query('main_secto=="U_Orthodox"')[['pop_20','pop_15']].sum().sum()*student_seminar_pre_from_age_15_20
        
        #מקדם תעסוקה לישיבות סמינרים
        forecast['student_yeshiva_vs_yeshiva_emp']=forecast['student_yeshiva']/forecast['emp_from_Yeshiva_student']
        forecast['seminar_vs_seminar_emp']=forecast['Seminar']/forecast['emp_from_Seminar_student']

        #פרופרוציה של כל ישיבה וסמינרים
        forecast['student_yeshiva_pre']=forecast['student_yeshiva']/forecast['student_yeshiva'].sum().item()
        forecast['student_seminar_pre']=forecast['Seminar']/forecast['Seminar'].sum().item()

        #בכמה כל ישיבה וסמינר גדל
        forecast['student_yeshiva_growth']=(forecast['student_yeshiva_pre']*total_student_yeshiva)/forecast['student_yeshiva']
        forecast['seminar_growth']=(forecast['student_seminar_pre']*total_student_seminar)/forecast['Seminar']

        #חישוב כמות תלמידי ישיבה וסמינר
        forecast['student_yeshiva']=forecast['student_yeshiva_pre']*total_student_yeshiva
        forecast['Seminar']=forecast['student_seminar_pre']*total_student_seminar

        #יצירת תעסוקה לישיבה וסמינר

        forecast['emp_from_Yeshiva_student']=forecast['student_yeshiva']/forecast['student_yeshiva_vs_yeshiva_emp']
        forecast['emp_from_Seminar_student']=forecast['Seminar']/forecast['seminar_vs_seminar_emp']
        
        #גידול פנימיות ביחס לגידול התלמידים
        forecast['yeshiva_dorms_pop_15']=forecast['yeshiva_dorms_pop_15']*forecast['student_yeshiva_growth']
        forecast['yeshiva_dorms_pop_20']=forecast['yeshiva_dorms_pop_20']*forecast['student_yeshiva_growth']
        forecast['yeshiva_dorms_pop_25']=forecast['yeshiva_dorms_pop_25']*forecast['student_yeshiva_growth']
        col_dorms_yeshiva=['yeshiva_dorms_pop_15', 'yeshiva_dorms_pop_20', 'yeshiva_dorms_pop_25']
        forecast['yeshiva_dorms_pop_sum']=forecast[col_dorms_yeshiva].sum(axis=1)

        forecast['Seminar_dorms_pop_15']=forecast['Seminar_dorms_pop_15']*forecast['seminar_growth']
        forecast['Seminar_dorms_pop_20']=forecast['Seminar_dorms_pop_20']*forecast['seminar_growth']
        forecast['Seminar_dorms_pop_25']=forecast['Seminar_dorms_pop_25']*forecast['seminar_growth']
        col_dorms_seminar=['Seminar_dorms_pop_15', 'Seminar_dorms_pop_20', 'Seminar_dorms_pop_25']
        forecast['seminar_dorms_pop_sum']=forecast[col_dorms_seminar].sum(axis=1)

        #יצירת כוללים
        forecast['kollim_demand']=(forecast['pop_20']*0.8+forecast['pop_25']*0.65+forecast['pop_30']*0.30+forecast['pop_35']*0.30+forecast['pop_40']*0.30+forecast['pop_45']*0.20+forecast['pop_50']*0.20+forecast['pop_55']*0.20+forecast['pop_60']*0.20)*0.5 #הכפלה בחצי בשביל לקבל אוכלוסיית גברים מעורכת

        forecast.loc[forecast['main_secto']!='U_Orthodox','kollim_demand']=0  #אל אף שאנחנו יודעים שיש כוללים באזורים שהם לא מוגדרים כחרדים

        forecast['add_from_kollim_demand']=0

        forecast.loc[(forecast['main_secto']=='U_Orthodox')&(forecast['kollim_demand']>forecast['student_yeshiva']),'add_from_kollim_demand']=forecast['kollim_demand']-forecast['student_yeshiva']

        forecast['student_yeshiva_and_kollim']=forecast['add_from_kollim_demand']+forecast['student_yeshiva']


        forecast['UO_Hi_Ed']=forecast['student_yeshiva_and_kollim']+forecast['Seminar']

        forecast['UNIVENRORTHFEMALE']=forecast['Univ_UO']/2+forecast['Seminar']
        forecast['UNIVENRORTHMALE']=forecast['Univ_UO']/2+forecast['student_yeshiva_and_kollim']

        forecast=forecast.fillna(0)

        forecast['emp_Education']=forecast['emp_from_student']+forecast['emp_from_Yeshiva_student']+forecast['emp_uni']

        emp_Education=forecast.query('main_secto!="Palestinian"')['emp_Education'].sum().item()

        #יצירת מקומות עבודה עוקב
        EMP_buffer=EMP_kibolet.set_geometry('geometry_buff').query('F{} > 0 & effects_okev==1'.format(y))

        EMP_buffer=EMP_buffer.dissolve()[['geometry_buff']]

        emp_buffer_taz=gpd.overlay(EMP_buffer,taz)

        emp_buffer_taz['emp_samll_area']=emp_buffer_taz.area

        emp_buffer_taz['emp_pre_from_taz']=emp_buffer_taz['emp_samll_area']/emp_buffer_taz['taz_area']

        taz_num_no_need_okev=emp_buffer_taz.loc[emp_buffer_taz['emp_pre_from_taz']>0.4].Taz_num.to_list()

        forecast['okev']=0

        forecast.loc[(~forecast['Taz_num'].isin(taz_num_no_need_okev))&(forecast['main_secto']!="Palestinia"),'okev']=1

        forecast.loc[forecast['main_secto']=="arabs_behined_seperation_wall",'okev']=1
        forecast['emp_okev']=0

        forecast.loc[(forecast['jew']==0)&(forecast['okev']==1),'emp_okev']=forecast['aprt']*0.3

        forecast.loc[(forecast['main_secto']=="U_Orthodox")&(~forecast['Muni_Heb'].isin(city_muni))&(forecast['okev']==1),'emp_okev']=forecast['aprt']*0.3

        forecast.loc[(forecast['main_secto']=="U_Orthodox")&(forecast['Muni_Heb'].isin(city_muni))&(forecast['okev']==1),'emp_okev']=forecast['aprt']*0.3

        forecast.loc[(forecast['main_secto']=="Jewish")&(forecast['Muni_Heb'].isin(city_muni))&(forecast['okev']==1),'emp_okev']=forecast['aprt']*0.2

        forecast.loc[(forecast['main_secto']=="Jewish")&(~forecast['Muni_Heb'].isin(city_muni))&(forecast['okev']==1),'emp_okev']=forecast['aprt']*0.2

        emp_okev=forecast['emp_okev'].sum().item()

        
        forecast=forecast.fillna(0)


        #הוספת סטודנטים ותלמידי ישיבה לאוכלוסייה
        forecast['pop_without_dorms_yeshiva']=forecast['pop']

        forecast['pop']=forecast['pop']+forecast['student_dorms']

        forecast['pop_20_just_from_aprt']=forecast['pop_20']

        forecast['pop_25_just_from_aprt']=forecast['pop_25']

        forecast['pop_20']=forecast['pop_20']+forecast['student_dorms']*0.6

        forecast['pop_25']=forecast['pop_25']+forecast['student_dorms']*0.4

        forecast['pop_15_just_from_aprt']=forecast['pop_15']

        forecast.loc[forecast['main_secto']=="U_Orthodox",'pop']=forecast['pop']+forecast['yeshiva_dorms_pop_sum']

        forecast.loc[forecast['main_secto']=="U_Orthodox",'pop_15']=forecast['pop_15']+forecast['yeshiva_dorms_pop_15']

        forecast.loc[forecast['main_secto']=="U_Orthodox",'pop_20']=forecast['pop_20']+forecast['yeshiva_dorms_pop_20']

        forecast.loc[forecast['main_secto']=="U_Orthodox",'pop_25']=forecast['pop_25']+forecast['yeshiva_dorms_pop_25']
        forecast=forecast.fillna(0)

        forecast['hh']=forecast['aprt']+forecast['student_dorms']/unvi_in_dorms

        forecast['hh']=forecast['hh']+forecast['yeshiva_dorms_pop_sum']

        #יצירת שאר מקומות עבודה
        forecast['work_age']=forecast[work_age].sum(axis=1)

        forecast['under_work_age']=forecast[under_work_age].sum(axis=1)

        forecast['over_work_age']=forecast[over_work_age].sum(axis=1)

        sector=['U_Orthodox','Jewish','Arab','arabs_behined_seperation_wall']

        for se in sector:

            #print (y)
            #print (se)
            work_age_factor_woman = work_factor.loc[(work_factor.index == se) & (work_factor['age'] == 'work_age') & (work_factor['gender'] == 'woman') & (work_factor['year'] == y), 'value'].item()
            work_age_factor_man = work_factor.loc[(work_factor.index == se) & (work_factor['age'] == 'work_age') & (work_factor['gender'] == 'man') & (work_factor['year'] == y), 'value'].item()
            under_work_age_factor_woman = work_factor.loc[(work_factor.index == se) & (work_factor['age'] == 'under_work_age') & (work_factor['gender'] == 'woman') & (work_factor['year'] == y), 'value'].item()
            under_work_age_factor_man = work_factor.loc[(work_factor.index == se) & (work_factor['age'] == 'under_work_age') & (work_factor['gender'] == 'man') & (work_factor['year'] == y), 'value'].item()
            over_work_age_factor_woman = work_factor.loc[(work_factor.index == se) & (work_factor['age'] == 'over_work_age') & (work_factor['gender'] == 'woman') & (work_factor['year'] == y), 'value'].item()
            over_work_age_factor_man = work_factor.loc[(work_factor.index == se) & (work_factor['age'] == 'over_work_age') & (work_factor['gender'] == 'man') & (work_factor['year'] == y), 'value'].item()
            
            #print('work_age_factor_woman:',work_age_factor_woman)


            forecast.loc[forecast['main_secto'] == se, 'pop_emp'] = forecast['work_age'] * pre_woman * work_age_factor_woman + forecast['work_age'] * pre_man * work_age_factor_man
            forecast.loc[forecast['main_secto'] == se, 'pop_emp'] = forecast['pop_emp'] + forecast['under_work_age'] * pre_woman * under_work_age_factor_woman + forecast['under_work_age'] * pre_man * under_work_age_factor_man
            forecast.loc[forecast['main_secto'] == se, 'pop_emp'] = forecast['pop_emp'] + forecast['over_work_age'] * pre_woman * over_work_age_factor_woman + forecast['over_work_age'] * pre_man * over_work_age_factor_man

        forecast['pop_emp_employed']=0

        forecast.loc[forecast['main_secto']=="U_Orthodox",'pop_emp_employed']=forecast['pop_emp']*0.96

        forecast.loc[forecast['main_secto']=="Jewish",'pop_emp_employed']=forecast['pop_emp']*0.97

        forecast.loc[forecast['jew']==0,'pop_emp_employed']=forecast['pop_emp']*0.96
        forecast=forecast.set_index('Taz_num')

        forecast['pop_emp_employed_out_of_jtmt_area']=forecast['pop_emp_employed']*forecast['commuting']

        emp_from_jtmt_area=forecast['pop_emp_employed'].sum().item()-forecast['pop_emp_employed_out_of_jtmt_area'].sum().item()

        #print('emp_from_jtmt_area:', emp_from_jtmt_area)

        emp_in_jtmt_area=emp_from_jtmt_area*pre_commuting_to_jtmt_area #יוממות פנימה מחוץ למרחב

        
        emp_in_jtmt_area_without_mobile=emp_in_jtmt_area*from_emp_to_place_of_work #מקדם מועסקים למקומות עבודה

        #print('emp_in_jtmt_area_without_mobile:',emp_in_jtmt_area_without_mobile)

        #print('emp_okev:',emp_okev)

        #print('emp_not_okev_current:',emp_not_okev_current)

        
        add_emp_not_okev=emp_in_jtmt_area_without_mobile-emp_Education-emp_okev-emp_not_okev_current

        #print('add_emp_not_okev:',add_emp_not_okev)

        add_emp_not_okev_kibolet=EMP_kibolet['F{}'.format(y)].sum().item()-emp_not_okev_current

        #print('add_emp_not_okev_kibolet:',add_emp_not_okev_kibolet)
                
        add_emp_not_okev_filling=add_emp_not_okev/add_emp_not_okev_kibolet

        #print('add_emp_not_okev_filling:',add_emp_not_okev_filling)

        if add_emp_not_okev_filling<1:
            EMP_kibolet['emp_current']=(EMP_kibolet['F{}'.format(y)]-EMP_kibolet['emp_current'])*add_emp_not_okev_filling+EMP_kibolet['emp_current']
        else:
            EMP_kibolet['emp_current']=EMP_kibolet['F{}'.format(y)]

        emp_not_okev_current=EMP_kibolet['emp_current'].sum().item()

        #print('emp_not_okev_current:',emp_not_okev_current)


        col=['Taz_num','ID','prec_from_id']

        df=EMP_kibolet_by_taz.reset_index()[col].merge(EMP_kibolet,on='ID',how='left')

        df['emp_current']=df['emp_current']*df['prec_from_id']

        forecast['emp_not_okev']=df.drop(columns=['geometry', 'geometry_buff']).pivot_table(index='Taz_num',aggfunc='sum')[['emp_current']]

        forecast=forecast.fillna(0)

        forecast['total_emp']=forecast['emp_not_okev']+forecast['emp_okev']+forecast['emp_Education']

        #הוספת תלמידי ישיבה באזורים לא חרדים לאוכלוסייה
        forecast.loc[forecast['main_secto']!="U_Orthodox",'pop']=forecast['pop']+forecast['yeshiva_dorms_pop_sum']

        forecast.loc[forecast['main_secto']!="U_Orthodox",'pop_15']=forecast['pop_15']+forecast['yeshiva_dorms_pop_15']

        forecast.loc[forecast['main_secto']!="U_Orthodox",'pop_20']=forecast['pop_20']+forecast['yeshiva_dorms_pop_20']

        forecast.loc[forecast['main_secto']!="U_Orthodox",'pop_25']=forecast['pop_25']+forecast['yeshiva_dorms_pop_25']

        #פיצול מקומות עבודה לפי ענף

        col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

        forecast=forecast.join(emp_category_type[col]).fillna(0)

        for i in col:
            forecast['{}'.format(i)]=forecast['{}'.format(i)]*(forecast['emp_not_okev']+forecast['emp_okev'])

        col=['Taz_num',
            'Muni_Heb',
            'commuting',
            'main_secto','sector_for_chinuc','jew',
            'TOA1','TOA2','TOA3',
            'Univ_SE','Univ_AR','Univ_UO',
            'student_dorms','univ','emp_uni',
            'student_yeshiva','emp_from_Yeshiva_student',
            'yeshiva_dorms_pop_15',
            'yeshiva_dorms_pop_20',
            'yeshiva_dorms_pop_25',
            'yeshiva_dorms_pop_sum',
            'Seminar_dorms_pop_15',
            'Seminar_dorms_pop_20',
            'Seminar_dorms_pop_25',
            'Seminar','emp_from_Seminar_student']

        forecast_current=forecast.reset_index()[col]

        locals()['forecast_{}_{}'.format(y,s)]=forecast

C:\Users\dpere\AppData\Local\Temp\ipykernel_31032\1662081919.py:223: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[  74.65321654  190.12997895  105.44359877   39.8396948   116.42400361
   39.58367189  182.59845829   22.47195498  203.61200551   72.94964825
  361.97932097  289.08107077  219.45559174  106.75987479  213.28812379
  283.84535533   61.65115322   70.33466349  268.73433919  107.31557417
   74.27025188  271.29515735  151.57961086  209.05886741  185.11233522
  328.22293152  461.49129218  226.74337781  235.89130624  118.17933223
  169.71651364  203.46743314  268.85259296  110.61100977  139.88805159
  107.64333487   73.15565844   80.64430166  642.49268658  444.70026933
  350.65516718  361.04562227   75.34166081  250.13124371  346.66009424
  241.7124242   376.50976392  127.97487884  304.91016139  265.59796535
  346.44126421  100.31670481   54.91386912  649.23947087  541.89445012
 1307.13656221   99.84677813  91

In [27]:
sen=['jtmt','iplan','bau']

for s in sen:
    if s=='jtmt':
        output_location=r'{}\JTMT'.format(forecast_version_folder_location)
        save_intermediates=r'{}\JTMT\Intermediates'.format(create_forecast_basic_future_folder_path)
    if s=='iplan':
        output_location=r'{}\IPLAN'.format(forecast_version_folder_location)
        save_intermediates=r'{}\iplan\Intermediates'.format(create_forecast_basic_future_folder_path)
    if s=='bau':
        output_location=r'{}\BAU'.format(forecast_version_folder_location)
        save_intermediates=r'{}\bau\Intermediates'.format(create_forecast_basic_future_folder_path)
        
    for y in years:

        taz=locals()['forecast_{}_{}'.format(y,s)]

        taz=taz.merge(taz_geo_info_constant,on='Taz_num',how='left')   

        #tazSector
        taz['tazSector']=1 #ערבי
        taz.loc[taz['main_secto']=='U_Orthodox','tazSector']=2
        taz.loc[taz['main_secto']=='Jewish','tazSector']=3
        taz.loc[taz['main_secto']=='Palestinian','tazSector']=4


        #PUMA
        poly_pumas=up_load_shp(r'{}\background_files\poly_pumas.shp'.format(create_forecast_basic_folder_path))
        col_old=['poly_puma',  'F3', 'F2', 'F1', 'geometry']
        col_new=['poly_puma',  '3', '2', '1', 'geometry']
        poly_pumas=drop_geo(change_cols_names(poly_pumas,col_old,col_new))
        pumas_by_poly_sector=poly_pumas.melt(id_vars='poly_puma',var_name='tazSector',value_name='PUMA')
        pumas_by_poly_sector['tazSector']=pumas_by_poly_sector['tazSector'].astype(int)
        taz=taz.merge(pumas_by_poly_sector,on=['poly_puma','tazSector'],how='left')

        taz.loc[taz['PUMA']==0,'PUMA']=999
        taz.loc[taz['pop']==0,'PUMA']=999
        taz.loc[taz['main_secto']=='Palestinian','PUMA']=999
        taz.loc[taz['jeru_metro']==0,'PUMA']=999

        
        col_needed=['Taz_num',
        'yosh',
        'jeru_metro',
        'jerusalem_',
        'main_secto',
        'hh',
        'pop',
        'pop_0',
        'pop_5',
        'pop_10',
        'pop_15',
        'pop_20',
        'pop_25',
        'pop_30',
        'pop_35',
        'pop_40',
        'pop_45',
        'pop_50',
        'pop_55',
        'pop_60',
        'pop_65',
        'pop_70',
        'pop_75up',
        'total_emp',
        'Indus',
        'Com_hotel',
        'Business',
        'Public',
        'emp_Education',
        'agri',
        'student',
        'univ',
        'UO_Hi_Ed',
        'pop_emp_employed',
        'slope',
        'Urban']

        col_new_name=['TAZ',
        'yosh',
        'in_jerusalem_metropolin',
        'jerusalem_city',
        'sector',
        'hh_total',
        'pop',
        'age0_4',
        'age5_9',
        'age10_14',
        'age15_19',
        'age20_24',
        'age25_29',
        'age30_34',
        'age35_39',
        'age40_44',
        'age45_49',
        'age50_54',
        'age55_59',
        'age60_64',
        'age65_69',
        'age70_74',
        'age75up',
        'emp_tot',
        'indus',
        'com_hotel',
        'business',
        'public',
        'education',
        'agri',
        'student',
        'univ',
        'UO_Hi_Ed',
        'pop_emp_employed',
        'slope',
        'urban']

      
        df=change_cols_names(taz, col_needed, col_new_name)

        col_to_int=['TAZ',
        'yosh',
        'in_jerusalem_metropolin',
        'jerusalem_city',
        'hh_total',
        'pop',
        'age0_4',
        'age5_9',
        'age10_14',
        'age15_19',
        'age20_24',
        'age25_29',
        'age30_34',
        'age35_39',
        'age40_44',
        'age45_49',
        'age50_54',
        'age55_59',
        'age60_64',
        'age65_69',
        'age70_74',
        'age75up',
        'emp_tot',
        'indus',
        'com_hotel',
        'business',
        'public',
        'education',
        'agri',
        'student',
        'univ',
        'UO_Hi_Ed',
        'pop_emp_employed',
        'slope',
        'urban']

        for c in col_to_int:
            df.loc[:, c] = df.loc[:, c].astype(int)

        df.sort_values(by='TAZ',ascending=True).to_excel(r'{}\BaseProjections{}_{}_{}.xlsx'.format(output_location,y,file_date,s),index=False)

        taz['DISTRICT']=999
        taz.loc[(taz['jew']==0)&(taz['main_secto']!='Palestinian'),'DISTRICT']=1
        taz.loc[(taz['main_secto']=='U_Orthodox')&(taz['in_jerusal']=='yes'),'DISTRICT']=2
        taz.loc[(taz['main_secto']=='Jewish')&(taz['in_jerusal']=='yes'),'DISTRICT']=3
        taz.loc[(taz['main_secto']=='Jewish')&(taz['in_jerusal']=='no')&(taz['jeru_metro']==1),'DISTRICT']=5
        taz.loc[(taz['main_secto']=='U_Orthodox')&(taz['in_jerusal']=='no')&(taz['jeru_metro']==1),'DISTRICT']=6
        taz.loc[taz['pop']==0,'DISTRICT']=999


        col_needed=['Taz_num','Agg_taz_nu','PUMA','DISTRICT','REGION','SCHOOLDIST']

        col_new_name=['TAZ','AGG_TAZ','PUMA','DISTRICT','REGION','SCHOOLDISTRICT']

        change_cols_names(taz, col_needed, col_new_name).sort_values(by='TAZ',ascending=True).to_excel(r'{}\puma{}_{}_{}.xlsx'.format(output_location,y,file_date,s),index=False)

        col_needed=['Taz_num',
        'Taz_num',
        'hh',
        'PUMA',
        'DISTRICT',
        'county',
        'area',
        'parktot',
        'majunivenr',
        'tazSector',
        'Indus',
        'Com_hotel',
        'Business',
        'Public',
        'emp_Education',
        'agri',
        'total_emp',
        'UOA1',
        'UOA2',
        'UOA3',
        'SEA1',
        'SEA2',
        'SEA3',
        'ARA1',
        'ARA2',
        'ARA3',
        'TOA1',
        'TOA2',
        'TOA3',
        'Univ_AR',
        'Univ_SE',
        'UNIVENRORTHMALE',
        'UNIVENRORTHFEMALE',
        'ieold',
        'superZone',
        'IEProp',
        'Taz1',
        'perScaled',
        'EIProp',
        'CITYCODE1',
        'CITYCODE2',
        'CITYCODE3',
        'CITYCODE4',
        'codeseq',
        'codeseqCon',
        'PaidBuffer',
        'Rest_EmpBu',
        'FreeBuffer',
        'SCHOOLDIST',
        'SCHOOLDIST',
        'highBusine',
        'searchtime',
        'walktime',
        'cost']


        col_new_name=['maz',
        'taz',
        'hh_total',
        'puma',
        'district',
        'county',
        'area',
        'parktot',
        'majunivenr',
        'tazSector',
        'Indus',
        'Com_hotel',
        'Off_Bsness',
        'Public',
        'Education',
        'Agri',
        'totemp',
        'UOA1',
        'UOA2',
        'UOA3',
        'SEA1',
        'SEA2',
        'SEA3',
        'ARA1',
        'ARA2',
        'ARA3',
        'TOA1',
        'TOA2',
        'TOA3',
        'UNIVENRARAB',
        'UNIVENRSEC',
        'UNIVENRORTHMALE',
        'UNIVENRORTHFEMALE',
        'ieold',
        'superZone',
        'IEProp',
        'Taz1',
        'perScaled',
        'EIProp',
        'CITYCODE1',
        'CITYCODE2',
        'CITYCODE3',
        'CITYCODE4',
        'codeseq',
        'codeseqCons',
        'PaidBuffer',
        'Rest_EmpBuffer',
        'FreeBuffer',
        'schDistrict',
        'schDistrictAgg',
        'highBusinessFlag',
        'searchtime',
        'walktime',
        'cost']

        

        df=change_cols_names(taz, col_needed, col_new_name)

        col_to_int=['maz',
                    'taz',
                    'hh_total',
                    'puma',
                    'district',
                    'county',
                    'area',
                    'parktot',
                    'majunivenr',
                    'tazSector',
                    'Indus',
                    'Com_hotel',
                    'Off_Bsness',
                    'Public',
                    'Education',
                    'Agri',
                    'totemp',
                    'UOA1',
                    'UOA2',
                    'UOA3',
                    'SEA1',
                    'SEA2',
                    'SEA3',
                    'ARA1',
                    'ARA2',
                    'ARA3',
                    'TOA1',
                    'TOA2',
                    'TOA3',
                    'UNIVENRARAB',
                    'UNIVENRSEC',
                    'UNIVENRORTHMALE',
                    'UNIVENRORTHFEMALE',
                    'superZone',
                    'Taz1',
                    'CITYCODE1',
                    'CITYCODE2',
                    'CITYCODE3',
                    'CITYCODE4',
                    'codeseq',
                    'codeseqCons',
                    'PaidBuffer',
                    'Rest_EmpBuffer',
                    'FreeBuffer',
                    'schDistrict',
                    'schDistrictAgg',
                    'highBusinessFlag',
                    'searchtime',
                    'walktime']
        
        for c in col_to_int:
            df.loc[:, c] = df.loc[:, c].astype(int)

        df.sort_values(by='maz',ascending=True).to_excel(r'{}\SED_{}_{}_{}.xlsx'.format(output_location,y,s,file_date),index=False)

        save_excel_path=r'{}\{}_forecast_{}_full_{}.xlsx'.format(save_intermediates,file_date,y,s)

        taz.to_excel(save_excel_path,index=False)